In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import requests
from pathlib import Path

## Class Bokapi
* See queryInfo : Bokapi.queryInfoDf<br>
More info : <a href="https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideStatCd&menuGroup=MENU000004&menuCode=MENU000024"> ECOS Open API</a>
* Get data : Bokapi.getData(indicatorIndex, startDate, endDate, periodUnit='default')<br>Type of 'indicatorIndex' is int or list (eg: [code1, code2, code3, code4])

In [2]:
class Bokapi :
    
    def __init__(self, key):
        self.key = key
        self.start = '1'
        self.end = '10000'
        self.dataPath = Path.cwd() / 'DataBokAPI'
        self.fileName = 'bokApiInfo.parquet'
        self.queryInfoDf = pd.read_parquet(self.dataPath / self.fileName)
    
    def getUrl(self) :
        edge_url =  '/'.join(self.codeList[1:])+'/'
        url = 'http://ecos.bok.or.kr/api/StatisticSearch/' + self.key + '/json/kr/' + self.start + '/' + self.end + '/' + self.codeList[0] + '/' \
                + self.periodUnit+ '/' + self.startDate + '/' + self.endDate + '/' + edge_url
        print(url)
        return url
    
    def setIndex(self, resultDf) :
        dateSeries = resultDf.reset_index().loc[:, 'index'].apply(lambda x: x[:-1] + "Q" + x[-1:])
        dateSeries = dateSeries.apply(lambda x : pd.Period(x, freq='M'))
        dateSeries = dateSeries.apply(lambda x : str(x)).tolist()
        dateSeries = pd.Series(dateSeries)
        dateSeries = pd.to_datetime(dateSeries, format='%Y-%m')
        resultDf = resultDf.set_index(dateSeries)
        
        return resultDf
        
    def getData(self, indicatorIndex, startDate, endDate, periodUnit='default') :
        
        # set self.codeList and resultColName
        if isinstance(indicatorIndex, int):
            series = self.queryInfoDf.iloc[indicatorIndex]
            self.codeList = series[['code1', 'code2', 'code3', 'code4']].dropna().tolist()
            resultColName=[series['통계지표']+ ' ' +series['단위'].join('()')]
            
        elif isinstance(indicatorIndex, list):
            self.codeList = indicatorIndex
            resultColName=[self.codeList[0]]

        # set date period for query 
        self.startDate = startDate
        self.endDate = endDate
        
        # set period_unit
        self.periodUnit = periodUnit
        if periodUnit=='default':
            self.periodUnit= self.queryInfoDf.iloc[indicatorIndex]['frequency1']
        
        # get url for query
        url = self.getUrl()
        
        # request data
        re = requests.get(url)
        j = re.json()
        data = j['StatisticSearch']['row']

        valueList = []
        timeList = []

        for item in data:
            value = item['DATA_VALUE']
            time = item['TIME']
            valueList.append(value)
            timeList.append(time)
        
        resultDf = pd.DataFrame(valueList, index=timeList, columns=resultColName)
        
        resultDf = self.setIndex(resultDf)
        
        return resultDf

In [3]:
bok = Bokapi(key='EDWTSFA1RXMHZMM9PQQN')

### Load 'Bok API info' parquet file

In [4]:
bok.queryInfoDf.iloc[0:6]

,통계구분,통계지표,시점,원자료,단위,code1,code2,code3,code4,frequency1,frequency2,frequency3
0,국민소득-경기-기업경영,"경제성장률(실질, 계절조정 전기대비)",20211,1.6,%,111Y055,10111,None,None,QQ,None,None
1,국민소득-경기-기업경영,"민간소비증감률(실질, 계절조정 전기대비)",20211,1.1,%,111Y055,10122,None,None,QQ,None,None
2,국민소득-경기-기업경영,"설비투자증감률(실질, 계절조정 전기대비)",20211,6.6,%,111Y055,10123,None,None,QQ,None,None
3,국민소득-경기-기업경영,"건설투자증감률(실질, 계절조정 전기대비)",20211,0.4,%,111Y055,10124,None,None,QQ,None,None
4,국민소득-경기-기업경영,"GDP(명목, 계절조정)",20204,"490,139.30",십억원,111Y012,1400,None,None,QQ,None,None
5,국민소득-경기-기업경영,GDP디플레이터,20204,104.416,2015=100,111Y046,1400,None,None,QQ,YY,None


In [41]:
startDate = '201901'
endDate = '202201'
returnDf = bok.getData(0, startDate, endDate, periodUnit= 'QQ')
returnDf.iloc[:,0] = returnDf.iloc[:,0].astype('float')
returnDf

http://ecos.bok.or.kr/api/StatisticSearch/EDWTSFA1RXMHZMM9PQQN/json/kr/1/10000/111Y055/QQ/201901/202201/10111/


,"경제성장률(실질, 계절조정 전기대비) (%)"
2019-01-01,-0.2
2019-04-01,1.0
2019-07-01,0.4
2019-10-01,1.3
2020-01-01,-1.3
2020-04-01,-3.2
2020-07-01,2.2
2020-10-01,1.1
2021-01-01,1.7
2021-04-01,0.8


In [8]:
startDate = '201901'
endDate = '202201'
returnDf = bok.getData(['111Y017', '10101'], startDate, endDate, periodUnit= 'QQ')
returnDf.iloc[:,0] = returnDf.iloc[:,0].astype('float')
returnDf.index

http://ecos.bok.or.kr/api/StatisticSearch/EDWTSFA1RXMHZMM9PQQN/json/kr/1/10000/111Y017/QQ/201901/202201/10101/


DatetimeIndex(['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01',
               '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01',
               '2021-01-01', '2021-04-01'],
              dtype='datetime64[ns]', freq=None)